In [21]:
import cvxpy as cp
import numpy as np

# Problem data.
m = 30
n = 20
np.random.seed(1)
A = np.random.randn(m, n)
b = np.random.randn(m)

# Construct the problem.
x = cp.Variable(n)
objective = cp.Minimize(cp.sum_squares(A @ x - b))
constraints = [0 <= x, x <= 1, cp.sum(x) == 1]
prob = cp.Problem(objective, constraints)

# The optimal objective value is returned by `prob.solve()`.
result = prob.solve()
# The optimal value for x is stored in `x.value`.

print(x.value)

[-6.39755587e-18 -8.43202002e-18  4.20497258e-19 -1.93725524e-18
 -5.70938710e-18  1.25673263e-01 -5.76443664e-18 -7.32510730e-19
  1.66909960e-01  5.07152051e-01 -4.41980455e-18 -3.20262829e-18
 -4.28020508e-18  2.00264725e-01 -3.40334378e-18 -1.16929317e-17
 -6.05430188e-18 -8.09229473e-18 -6.28897713e-18 -1.27713008e-17]


In [4]:
class ConstrainedRegression:
    def __init__(self):
        self.constraints = []

    def add_constraint(self, x, ge=None, le=None):
        if ge:
            self.constraints.append(("ge", x, ge))
        if le:
            self.constraints.append(("le", x, le))

    def fit(self, X, y):
        w = cp.Variable(X.shape[1])
        b = cp.Variable(1)
        objective = cp.Minimize(cp.sum_squares(X @ w + b - y))
        self.constraints_ = []
        for c in self.constraints:
            if c[0] == 'ge':
                self.constraints_.append((c[1] @ w + b) >= c[2])
            if c[0] == 'le':
                self.constraints_.append((c[1] @ w + b) <= c[2])
        self.prob_ = cp.Problem(objective, self.constraints_)
        self.prob_.solve()
        self.coef_ = w.value
        self.intercept_ = b.value
        return self

    def predict(self, X):
        return X @ self.coef_ + self.intercept_

In [5]:
from sklearn.datasets import make_regression

X, y = make_regression(n_samples=10000, random_state=42)

In [6]:
y[0]

-213.2662757396957

In [7]:
y[0] += 250

In [8]:
from sklearn.linear_model import LinearRegression

lin = LinearRegression().fit(X, y)

In [9]:
con = ConstrainedRegression().fit(X, y)

When we do not consider constraints ... the predictions are exactly the same!

In [10]:
lin.predict(X)[:10]

array([-210.78479202,  271.9507549 , -292.53195041,   58.16860197,
        171.13497985, -128.24130413,   39.81402271,  -75.26708694,
       -380.03185675, -245.61790134])

In [11]:
con.predict(X)[:10]

array([-210.78479202,  271.9507549 , -292.53195041,   58.16860197,
        171.13497985, -128.24130413,   39.81402271,  -75.26708694,
       -380.03185675, -245.61790134])

In [12]:
lin.coef_[:10], lin.intercept_

(array([ 0.04116305, -0.00527586,  0.00994886,  0.00997819, -0.05529895,
        -0.02320834,  0.01363836, -0.04392767, -0.01102636, -0.01395106]),
 0.025091612969682764)

In [13]:
con.coef_[:10], con.intercept_

(array([ 0.04116305, -0.00527586,  0.00994886,  0.00997819, -0.05529895,
        -0.02320834,  0.01363836, -0.04392767, -0.01102636, -0.01395106]),
 array([0.02509161]))

But lets now say that that first prediction is actually super duper important.

In [14]:
y[0]

36.733724260304314

In [15]:
con = ConstrainedRegression()
con.add_constraint(X[0], le=50, ge=30)

In [16]:
con.fit(X, y)

In [17]:
con.constraints_

[Inequality(Constant(CONSTANT, NONNEGATIVE, ())),
 Inequality(Expression(AFFINE, UNKNOWN, (1,)))]

In [18]:
lin.predict(X)[:10]

array([-210.78479202,  271.9507549 , -292.53195041,   58.16860197,
        171.13497985, -128.24130413,   39.81402271,  -75.26708694,
       -380.03185675, -245.61790134])

In [19]:
con.predict(X)[:10]

array([  30.        ,  343.77110003, -267.90380904,   38.03676497,
        153.27388416, -149.8483624 ,   67.42506185,  -47.94535284,
       -367.61234668, -258.55333212])

So this is pretty interesting. 

We do not need to resort to sample weights here. We can totally drop the hammer and force the behavior out with a constraint.